## Task 3
Encode and decode a message "Father Christmas does not exist".

LSB steganography technique will be used to encode and decode secret message.

#### Structure:

- create secrete message and key
- create encoding algorithm
- create decoding algorithm
- display result of decoded message
- references

In [1]:
# import libraries
import wave
import array
import random
import os

#### Secret message and encryption key

In [2]:
secret_message = 'Father Christmas does not exist'
encryption_key = 94874 # can be changed to any INT
bits = 2 # even numbers only

#### Encoding Algorithm

In [3]:
original_file = 'audio/Ex3_sound5.wav'
new_file = 'audio/secret_message_file.wav'

def encoding_algorithm(original_file, secret_message, encryption_key, bits=1):
    # secret message from text to binary conversion 
    message_in_binary = ''.join(format(ord(i), '08b') for i in secret_message)
    try:
        # try opening file
        with wave.open(original_file, 'rb') as file:
            # print confirmation message
            print(f"The {original_file} was succefully opened.")
            # read frames and get parameters
            file_data = array.array('h', file.readframes(file.getnframes()))
            parameters = file.getparams()
    
    # raise an error   
    except IOError:
        print(f"The {original_file} can not be opened. Please check if {original_file} exists in specified location.")
        return
    
    # random number generator initialization
    random_number_generator = random.Random()
    random_number_generator.seed(encryption_key)

    # random indices
    indices = random_number_generator.sample(range(len(file_data)), len(message_in_binary) // bits)
    indices.sort() # sort in ascending order

    # encode secret message into file
    # loop over message in binary and i++bits 
    for index_of_bit in range(0, len(message_in_binary), bits):
        # based on random number generator get random frame 
        audio_frame = file_data[indices[index_of_bit // bits]]
        
        # loop over bits
        for position_of_bit in range(bits):
            # modify bits in audio frame by replacing with bits from message in binary 
            audio_frame = (audio_frame & ~(1<<position_of_bit)) | (int(message_in_binary[index_of_bit+position_of_bit])<<position_of_bit)
        # save with encoded message bits to position in the file 
        file_data[indices[index_of_bit // bits]] = audio_frame

# I would normally put a check for if the file already exists, not to override it, 
# but for grading purposes it makes it easier to have the file overriden this way 
# it doesnt need to be deleted before running the program
    try:
        # try saving encoded audio to a new file
        with wave.open(new_file, 'wb') as file:
            # print confirmation message
            print(f"Opened the {new_file} for writting.")
            file.setparams(parameters)
            file.writeframes(file_data.tobytes())
        # print confirmation message
        print(f"Closed the {new_file} after writting.")
    # raise an error
    except IOError:
        print(f"The {new_file} can not be open. Please check the {new_file} path.")
        return

encoding_algorithm(original_file, secret_message, encryption_key, bits)

The audio/Ex3_sound5.wav was succefully opened.
Opened the audio/secret_message_file.wav for writting.
Closed the audio/secret_message_file.wav after writting.


#### Decoding Algorithm

In [4]:
def decoding_algorithm(new_file, text_length, encryption_key, bits=1):
    try:
        # try opening file
        with wave.open(new_file, 'rb') as file:
            # print confirmation message
            print(f"The {new_file} was succefully opened.")
            # read frames
            file_data = array.array('h', file.readframes(file.getnframes()))
         
    # raise an error   
    except IOError:
        print(f"The {new_file} can not be opened. Please check if {new_file} exists in specified location.")
        return
        
   # random number generator initialization
    random_number_generator = random.Random()
    random_number_generator.seed(encryption_key)

    # random indices
    indices = random_number_generator.sample(range(len(file_data)), text_length * 8 // bits)
    indices.sort() # sort in ascending order

    # decode message
    message_in_binary = ''
    # iterate through index of bits from zero to 8 (each character = 8 bits)
    for index_of_bit in range(0, text_length * 8, bits):
        # get audio at index where secret message is located
        audio_frame = file_data[indices[index_of_bit // bits]]
        
        # extract bits
        for position_of_bit in range(bits):
            # move audio frame to position of bit, extract bit, convert it to string and add it to message in binary
            message_in_binary += ''.join(str((audio_frame >> position_of_bit) & 1))
   
   
    # message in binary split into parts of 8 bits -> converts to INT -> converts INT to ASCII char
    # combines all characters together by using join()
    secret_message = ''.join(chr(int(message_in_binary[index_of_bit:index_of_bit+8], 2)) 
                             for index_of_bit in range(0, len(message_in_binary), 8))
    return secret_message

#### Display Result of Decoding

In [5]:
print('Decoded Secret Message: ' + decoding_algorithm(new_file, len(secret_message), encryption_key, bits))

The audio/secret_message_file.wav was succefully opened.
Decoded Secret Message: Father Christmas does not exist


### References:
- https://en.wikipedia.org/wiki/Steganography
- https://iaeme.com/MasterAdmin/Journal_uploads/IJCET/VOLUME_12_ISSUE_3/IJCET_12_03_001.pdf
- https://www.researchgate.net/publication/338029976_An_Overview_of_Digital_Audio_Steganography
- https://docs.python.org/3/library/wave.html
- https://docs.python.org/3/library/random.html
- https://docs.python.org/3/library/functions.html#chr
- https://docs.python.org/3/library/stdtypes.html#str.join
- https://docs.python.org/3/library/stdtypes.html#string-methods